In [1]:
import os

os.chdir("../")
%pwd

'c:\\Users\\anfe1\\OneDrive\\Escritorio\\carpetaANFE\\AI_ML_Projects\\Text-Summarization'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        config_file_path = CONFIG_FILE_PATH
        params_file_path = PARAMS_FILE_PATH

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [4]:
import os
import zipfile
import urllib.request as request

from textSummarization.logging import logger
from textSummarization.utils.common import get_size

class DataIngestion:
    def __init__(self, config:DataIngestionConfig) -> None:
        self.config = config

    def download_file(self) -> None:
        """Download files from a given URL
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info:\n{headers}")
        else:
            logger.info(f"File already exists. File size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self) -> None:
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [5]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-15 15:26:22,660] 28 common - INFO - yaml file config\config.yml loaded successfully
[2024-03-15 15:26:22,664] 28 common - INFO - yaml file params.yml loaded successfully
[2024-03-15 15:26:22,667] 46 common - INFO - Created directory at: artifacts
[2024-03-15 15:26:22,669] 46 common - INFO - Created directory at: artifacts/data_ingestion
[2024-03-15 15:26:24,713] 20 2814335153 - INFO - artifacts/data_ingestion/data.zip downloaded with the following info:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7CAC:48A6:2ABE3:37761:65F4A51D
Accept-Ranges: bytes
Date: Fri, 15 Mar 2024 20:26:24 GMT
Via: 1.1 varnish
X-Served-By: cache-bog226